In [1]:
!head './data/X_train.csv'

In [2]:
!head './data/y_train.csv'

In [3]:
import numpy as np
import pandas as pd

/home/alex/miniconda3/envs/shell/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [27]:
df_train = pd.read_csv('data/X_train.csv')
df_train_y = pd.read_csv('data/y_train.csv',header=None)
print(df_train.shape[0])
df_train.head()

11173


,m_or_f,person_attributes,seat,other_person_location,other_factor_1,other_factor_2,other_factor_3,age_in_years
0,m,driving,front_left,NaN,NaN,NaN,NaN,50
1,m,on_bike,not_in_vehicle,in_unmarked_intersection,pedestrial_in_middle_of_street,NaN,NaN,42
2,f,driving,front_left,NaN,NaN,NaN,NaN,42
3,f,passenger,front_right,NaN,NaN,NaN,NaN,52
4,m,driving,front_left,NaN,NaN,NaN,NaN,45


In [5]:
df_train.describe()

,age_in_years
count,11173.000000
mean,46.673409
std,4.552841
min,40.000000
25%,43.000000
50%,46.000000
75%,51.000000
max,55.000000


In [6]:
df_train.isna().sum()

m_or_f                       1
person_attributes            0
seat                         0
other_person_location     9869
other_factor_1           10248
other_factor_2           10839
other_factor_3           11067
age_in_years                 0
dtype: int64

In [14]:
for col in df_train.columns:
    print("\n",col)
    print(df_train[col].value_counts())


 m_or_f
m    8202
f    2970
Name: m_or_f, dtype: int64

 person_attributes
driving                          8501
passenger                        1362
on_foot                          1084
on_bike                           178
in_stopped_vehicle                 23
other_on_foot                      12
unknown_in_other_vehicle_type       7
unknown_in_vehicle                  6
Name: person_attributes, dtype: int64

 seat
front_left                   8501
not_in_vehicle               1304
front_right                  1169
second_middle                  43
front_middle                   36
cargo_area                     34
truck_cab_sleeper_area         23
cargo_open_area                20
riding_outside_of_vehicle      15
front_unknown                   6
third_right                     5
second_other                    5
middle_third                    3
third_unknown                   2
in_trailer                      2
cargo_area_unknown              2
second_unknown                 

In [16]:
import pickle
import json
import pandas as pd
# this category_encoders package is not part of the
# sklearn core packages, it is a 3rd party but makes
# our lives a lot easier because it can deal with encoding
# strings whereas sklearn's OneHotEncoder cannot
import category_encoders
from sklearn.preprocessing import Imputer, FunctionTransformer, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

In [28]:
pipeline = make_pipeline(
    category_encoders.OneHotEncoder(handle_unknown='impute'),
    Imputer(strategy='mean'),
    LogisticRegression(),
)
pipeline.fit(df_train, df_train_y)

/home/alex/miniconda3/envs/shell/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=['m_or_f', 'person_attributes', 'seat', 'other_person_location', 'other_factor_1', 'other_factor_2', 'other_factor_3'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)), ('imputer', Imputer(axis=...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [32]:
df_train.iloc[23].to_dict()

{'m_or_f': 'm',
 'person_attributes': 'driving',
 'seat': 'front_left',
 'other_person_location': nan,
 'other_factor_1': nan,
 'other_factor_2': nan,
 'other_factor_3': nan,
 'age_in_years': 44}

In [42]:
new_obs_str = '{"m_or_f": "m", "person_attributes": "driving", "seat": "front_left", "other_person_location":null, "other_factor_1":null, "other_factor_2":null , "other_factor_3":null , "age_in_years": 44}'

In [43]:
new_obs_dict = json.loads(new_obs_str)

In [ ]:
new_obs_dict

In [52]:
obs = pd.DataFrame([new_obs_dict], columns = df_train.columns)
obs = obs.astype(df_train.dtypes)

pipeline.predict_proba(obs)

array([[0.40257301, 0.59742699]])

# Serialization

In [56]:
with open('columns.json', 'w') as fh:
    json.dump(df_train.columns.tolist(), fh)
    
with open('dtypes.pickle', 'wb') as fh:
    pickle.dump(df_train.dtypes, fh)
    
from sklearn.externals import joblib
joblib.dump(pipeline, 'pipeline.pickle') 

['pipeline.pickle']